In [2]:
# This notebook will guide you to make correct format of Huggingface dataset, in proper parquet format and visualizable in Huggingface dataset hub.
# We will take the example of the dataset "Otter-AI/MMVet" and convert it to the proper format.

In [3]:
import json

with open('../tmp/batch.json') as f:
    data = json.load(f)

In [4]:
data_new = {'score': {}, 'pair':{}, 'refer':{}}

for key, val in data.items():
    ids = list(val.keys())
    data_new['score'][key] = {id: val[id] for id in ids[:len(ids) // 2]}
    data_new['pair'][key] = {id: val[id] for id in ids[len(ids) // 2: len(ids) // 2 + len(ids) // 4]}
    data_new['refer'][key] = {id: val[id] for id in ids[len(ids) // 2 + len(ids) // 4:]}

In [5]:
data.keys()

dict_keys(['cvrr_continuity_and_object_instance_count', 'cvrr_fine_grained_action_understanding', 'cvrr_interpretation_of_social_context', 'cvrr_interpretation_of_visual_context', 'cvrr_multiple_actions_in_a_single_video', 'cvrr_non_existent_actions_with_existent_scene_depictions', 'cvrr_non_existent_actions_with_non_existent_scene_depictions', 'cvrr_partial_actions', 'cvrr_time_order_understanding', 'cvrr_understanding_emotional_context', 'cvrr_unusual_and_physically_anomalous_activities', 'youcook2_val', 'vatex_test', 'videochatgpt_gen'])

In [6]:
# from datasets import Dataset, Features, Value, Image
# import pandas as pd
# from tqdm import tqdm
# import os

# # Define the features for the dataset
# features = Features(
#     {
#         "id": Value(dtype="string"),
#         "Q": Value(dtype="string"),
#         "A": Value(dtype="string"),
#         "DimensionName": Value(dtype="string"),
#         "model": Value(dtype="string"),
#     }
# )

df_items = {
    "id": [],
    "Q": [],
    "A": [],
    "DimensionName": [],
    "model": [],
}

In [13]:
for dataset, all_content in data_new['score'].items():
    if 'cvrr_' in dataset:
        description_root = f"/work/LAS/wzhang-lab/mingl/cache/hub/{dataset[len('cvrr_'):]}/annotations_{dataset[len('cvrr_'):]}.json"
        print(description_root)
    else:
        description_root = ''
    with open(description_root) as f:
        description = json.load(f)
    for id, content in all_content.items():
        for model, response in content.items():
            id = int(id)
            q = description[id]['Q']
            a = response
            Q = f"""Here is a question and a response related to the video, you need to rate the response on a scale (0-5) based on the video:
                    Question:{q}
                    Answer from Assitant: {a}
                    You review: 
                """   
            df_items["id"].append(id)
            df_items["Q"].append(Q)
            df_items["DimensionName"].append(description)
            df_items["model"].append(model)

/Volumes/work/LAS/wzhang-lab/mingl/cache/cvrr-es/CVRR-ES/continuity_and_object_instance_count/annotations_continuity_and_object_instance_count.json
/Volumes/work/LAS/wzhang-lab/mingl/cache/cvrr-es/CVRR-ES/fine_grained_action_understanding/annotations_fine_grained_action_understanding.json
/Volumes/work/LAS/wzhang-lab/mingl/cache/cvrr-es/CVRR-ES/interpretation_of_social_context/annotations_interpretation_of_social_context.json
/Volumes/work/LAS/wzhang-lab/mingl/cache/cvrr-es/CVRR-ES/interpretation_of_visual_context/annotations_interpretation_of_visual_context.json
/Volumes/work/LAS/wzhang-lab/mingl/cache/cvrr-es/CVRR-ES/multiple_actions_in_a_single_video/annotations_multiple_actions_in_a_single_video.json
/Volumes/work/LAS/wzhang-lab/mingl/cache/cvrr-es/CVRR-ES/non_existent_actions_with_existent_scene_depictions/annotations_non_existent_actions_with_existent_scene_depictions.json
/Volumes/work/LAS/wzhang-lab/mingl/cache/cvrr-es/CVRR-ES/non_existent_actions_with_non_existent_scene_depict

FileNotFoundError: [Errno 2] No such file or directory: ''

In [15]:
df_items.head()

AttributeError: 'dict' object has no attribute 'head'

In [ ]:
dataset = Dataset.from_pandas(df_items, features=features)

In [ ]:
hub_dataset_path = "lmms-lab/VideoDetailDescription"
dataset.push_to_hub(repo_id=hub_dataset_path, split="test")